1. Simulate noisy (uniform, multiplicative) measurements from model with "true" parameters
2. Run Levenberg-Marquardt least squares optimisation to find "best fit" parameters (note the mismatch between noise and cost function types)
3. Compute log Hessian around best fit to 

Rinse and repeat

In [1]:
# Helper functions

# Get a timestamp in useful format
timestamp()=Dates.format(now(),"_yyyymmddTHHMMSSss")

timestamp (generic function with 1 method)

### Simulate data, look at both assays and add noise in each separately (such that relative error is 10% or 1%)

In [2]:
# Simulate data points with noise
include("../BK_functions/bk_setup_script.jl")
ydata_noiseless = BK_simulator(phi0, hcat(x_grid...), model_id=0) # Create dataset
#ydata_orig = ydata_noiseless.*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative)
#ydata_log = log(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative, but in log space assay)

ydata_orig = ydata_noiseless.*(1+(2*rand(size(ydata_noiseless))-1)*0.01) # Add up to 1% noise (uniform, multiplicative)
ydata_log = log(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.01) # Add up to 1% noise (uniform, multiplicat

cur_time = timestamp();

## Fit noisy original assay and compute Hessian around best fit

In [4]:
# Optimise phi to fit the noisy data well
using LsqFit

# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0;
    else
        phi_init = phi0.*(1+(2*rand(size(phi0))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x), hcat(x_grid...), ydata_orig, log(phi_init));
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_orig = exp(all_fits[findmin(all_costs)[2]].param);


# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = BK_simulator(phi, x, model_id=0)[:]
data_vec = ydata_orig;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_orig = ForwardDiff.hessian(g_cost, log(phi1_orig)); #log hessian
D_orig, V_orig = eig(hess_orig)

([244.88240390831464,14.30888443756489,0.03964609913259878,2.474570214106106e-9,-3.734258987805233e-10,-2.966456754902944e-12,-3.2864393976147196e-25,2.628009341697385e-16],
8x8 Array{Float64,2}:
 -0.228481     -0.0738031    -0.00284093   …  -1.67328e-7    0.000314163
  1.06611e-16   6.5773e-17   -5.58496e-15     -0.000502394   1.0        
 -0.106677     -0.229322      0.11656         -3.06596e-10  -2.1108e-6  
  8.39208e-27  -1.30567e-22   4.16085e-16      1.0           6.05244e-8 
 -0.0186744     0.0323736    -0.701606        -7.14751e-9   -5.9696e-7  
 -0.326841      0.925769      0.0715799    …  -2.71974e-10   6.88489e-8 
 -0.910461     -0.288162     -0.0108855        4.21353e-8   -7.86084e-5 
  0.0174016    -0.0288457     0.699222        -7.11681e-9   -2.01516e-7 )

## Fit noisy log assay and compute Hessian around best fit

In [ ]:
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0;
    else
        phi_init = phi0.*(1+(2*rand(size(phi0))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> log(abs(BK_simulator(exp(p), x))), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = log(BK_simulator(phi, x, model_id=0)[:])
data_vec = ydata_log;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_log = ForwardDiff.hessian(g_cost, log(phi1_log)); #log hessian of log assay
D_log, V_log = eig(hess_log)

In [ ]:
#save results
save("saves/figure_A1_hess"*cur_time*".jld", "hess_orig", hess_orig, "D_orig", D_orig,
"hess_log", hess_log, "D_log", D_log, "phi1_orig", phi1_orig, "phi1_log", phi1_log, "ydata_orig", ydata_orig, "ydata_log", ydata_log)

In [ ]:
[log(sort(abs(D_orig))) log(sort(abs(D_log)))]

### Do the full reduction

include("../BK_functions/bk_setup_script.jl")


# Run MBAM from those parameters
cur_time = timestamp();

#outp = load("saves/figure_A1_20161108T15185000.jld", "outp") #Step 1
outp = load("saves/figure_A1_20161108T17280800.jld", "outp") #Step1-2


# First iteration:
outp = MBAM(BK_simulator, phi1, x_grid, model_iters=[70, 71, 72], 
move_dir=[-1, -1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce, data_vec=ydata, prevRunResults=outp)

using JLD
save("saves/figure_A1"*cur_time*".jld", "outp", outp, "phi1", phi1, "ydata", ydata)

# Run MBAM from those parameters

# First iteration:
outp_log10 = MBAM(BK_simulator, phi1, x_grid, model_iters=[51], 
move_dir=[-1], boundary_time=10, verbose=1, reduce_func=BK_reduce, data_vec=log10(ydata))

using JLD
save("saves/figure_A1_log"*cur_time*".jld", "outp_log10", outp_log10, "phi1", phi1, "ydata", ydata)